In [ ]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup as bs

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

df = pd.read_csv("../0_data/legal_info.csv")

# 생성한 법정동 코드 고유값 지정
LAWD_CD_list = df['법정동시군구코드'].unique()

# API키 세팅
api_key = '발급받은 API키'

column_nm = [
  '거래금액', '거래유형', '건축년도', '년', '대지면적', '법정동', '연면적', '월',
 '일', '주택유형', '중개사소재지', '지번', '지역코드', '해제사유발생일', '해제여부'
 ]

for i in range(len(LAWD_CD_list)):
    
    # 서비스 URL
    url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcSHTrade"

    year_month_key = "202101"
    
    # 요청변수 파라미터 설정     
    params = "?" + \
            "ServiceKey=" + api_key + "&" + \
            "pageNo=" + "1" + "&" + \
            "numOfRows=" + "9999" + "&" + \
            "LAWD_CD=" + str(LAWD_CD_list[i])  + "&" + \
            "DEAL_YMD=" + year_month_key   

    # requests, BeautifulSoup 라이브러리를 사용한 데이터 수집
    res = requests.get(url + params)
    soup = bs(res.text, 'xml')
    items = soup.find_all('item')


    total = pd.DataFrame()
    # 수집된 데이터를 Pandas 데이터 프레임 형식으로 변환

    for k in range(len(items)):
        df_raw = []
        for j in column_nm:
#             print(LAWD_CD_list[i],k,j) # 에러 파악 여부 
            try:
                items_data = items[k].find(j).text
                df_raw.append(items_data)
            except:
                items_data = "존재하지 않음"
                df_raw.append(items_data)
        df = pd.DataFrame(df_raw).T
        df.columns = column_nm
        
        total = pd.concat([total, df])

    total.columns = column_nm

# total.head().T

directory = "data/sh_trade"
if not os.path.exists(directory):
    os.makedirs(directory)
    
file_name = '0_data/sh_trade/sh_trade_sample_' + '202101' + ".csv"
total.to_csv(file_name, index = False)